In [104]:
import pandas as pd

file_path = 'halflabelleddataset.csv'
df = pd.read_csv(file_path)

df['Sr. No'] = df['Sr. No'].fillna(method='ffill')
#df['Questions'] = df['Questions'].fillna('')
grouped_df = df.groupby('Sr. No')


required_columns = ['Data'] + [f'Unnamed: {i}' for i in range(3, 10)]
# Dictionary to store the processed DataFrames, keyed by 'Sr. No'
processed_groups = {}

for name, group in grouped_df:
    missing_columns = [col for col in required_columns if col not in group.columns]
    if missing_columns:
        print(f"Missing columns in group {name}: {missing_columns}")
        continue
    
    filtered_group = group[required_columns]
    filtered_group.fillna('', inplace=True)
    processed_groups[name] = filtered_group

df.drop(columns=required_columns, inplace=True)
new_df = df.groupby('Sr. No').first()

for i in range(1, len(processed_groups) + 1):
    d = processed_groups[i]
    new_columns = d.iloc[0]
    d.columns = new_columns
    d = d.drop(d.index[0])
    d.reset_index(drop=True, inplace=True)
    d.replace("", float("NaN") , inplace=True) 
    d.dropna(how='all', axis=1, inplace=True) 
    d = d.drop(d.index[-1])
    processed_groups[i] = d
    
new_df = new_df.assign(Data=processed_groups.values())
df = new_df

C:\Users\bavis\AppData\Local\Temp\ipykernel_20576\1077221503.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_group.fillna('', inplace=True)
C:\Users\bavis\AppData\Local\Temp\ipykernel_20576\1077221503.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_group.fillna('', inplace=True)
C:\Users\bavis\AppData\Local\Temp\ipykernel_20576\1077221503.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_group.fi

In [139]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor

X_text = df['Questions']
X_numeric = df['Data'].apply(lambda x: x.shape[0])
X_numeric_2 = df['Data'].apply(lambda x: x.shape[1])
y_type = df['Type']

# Vectorize text features
vectorizer = TfidfVectorizer()
X_text_vectorized = vectorizer.fit_transform(X_text)

# Combine text and numeric features
#print(pd.DataFrame(X_text_vectorized.toarray()).shape)
#X_combined = pd.concat([pd.DataFrame(X_text_vectorized.toarray()), X_numeric, X_numeric_2], axis=1)
#X_combined = X_combined[:49]
#X_combined = X_text_vectorized
# Split data into training and testing sets
X_train, X_test, y_type_train, y_type_test = train_test_split(X_text_vectorized, y_type, test_size=0.1)

type_model = RandomForestClassifier()
type_model.fit(X_train, y_type_train)

type_score = type_model.score(X_test, y_type_test)
print(f"Type Model Score: {type_score}")


Type Model Score: 1.0


In [159]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor

X_text = df['Questions']
y_alpha = df['Alpha']

# Vectorize text features
vectorizer = TfidfVectorizer()
X_text_vectorized = vectorizer.fit_transform(X_text)

X_combined = X_text_vectorized
# Split data into training and testing sets
X_train, X_test, y_alpha_train, y_alpha_test = train_test_split(X_combined, y_alpha, test_size=0.1)

# Train separate models for Alpha and Type predictions
alpha_model = RandomForestRegressor()
alpha_model.fit(X_train, y_alpha_train)

# Evaluate models
alpha_score = alpha_model.score(X_test, y_alpha_test)

print(f"Alpha Model Score: {alpha_score}")

Alpha Model Score: 0.980583333333333


In [164]:
X_text_sample = ["The sales increase (in %) for different product features in various markets:Perform a two-way ANOVA to determine if product feature and market affect sales increase at α = 0.01."]
X_text_vectorized_sample = vectorizer.transform(X_text_sample)

# Predict 'Alpha' values for the sample input
alpha_predictions_sample = alpha_model.predict(X_text_vectorized_sample)
print("Predicted Alpha:")
print(alpha_predictions_sample.round(2))

# Predict 'Type' values for the sample input
type_predictions_sample = type_model.predict(X_text_vectorized_sample)
print("Predicted Type:")
print(type_predictions_sample)

type_predictions_proba = type_model.predict_proba(X_text_vectorized_sample)
type_confidence = type_predictions_proba.max(axis=1)
print("Predicted Type with Confidence:")
for prediction, confidence in zip(type_predictions_sample, type_confidence):
    print(f"{prediction} with confidence {confidence:.2f}")

Predicted Alpha:
[0.01]
Predicted Type:
['Two Way FD']
Predicted Type with Confidence:
Two Way FD with confidence 0.51
